In [1]:
from IPython.display import display, HTML
import pandas as pd
import math
import datetime
import json
import requests
import numpy as np
import re
import jellyfish as jf
import geocoder
import time
import xgboost as xgb
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
import re
import matplotlib.pyplot as plt
import subprocess
import os.path
from subprocess import PIPE,Popen
from collections import defaultdict
from scipy import sparse
from sklearn.decomposition import TruncatedSVD
import pickle

import warnings
warnings.filterwarnings('ignore')

In [2]:
numberAttributes = 12

In [3]:
dirty = pd.read_csv("/home/felix/BlackOak/List_A/inputDB.csv")
clean = pd.read_csv("/home/felix/BlackOak/List_A/groundDB.csv", dtype={'SSN': object, 'ZIP': object})

In [4]:
distinct_values_per_column = []
for column in range(len(dirty.columns)):
    distinct_values_per_column.append(dict())

max_length = [0.0] * len(dirty.columns)
sum_length = np.array([0] * len(dirty.columns))
N = len(dirty)

for index, row in dirty.iterrows():
    for column in range(len(dirty.columns)):
        value = row[dirty.columns[column]]
        distinct_values_per_column[column][value] = distinct_values_per_column[column].get(value, 0) + 1

        
        max_length[column] = max (max_length[column], len(str(value)))
        sum_length[column] += len(str(value))
avg_length = sum_length / float(N)

for column in range(len(dirty.columns)):
    for key, value in distinct_values_per_column[column].iteritems():
        distinct_values_per_column[column][key] = value / float(N)

In [5]:
print max_length

[7, 13, 13, 21, 44, 21, 17, 29, 26, 41, 11, 9]


In [6]:
number_distinct_values_per_column = []
for column in range(len(dirty.columns)):
    number_distinct_values_per_column.append(len(distinct_values_per_column[column]))
print number_distinct_values_per_column

[94306, 15048, 681, 31917, 84228, 12695, 5070, 18825, 14558, 9735, 47358, 13608]


In [7]:
ground_truth = dirty.values != clean.values #all real errors
print ground_truth

[[False  True False ..., False  True False]
 [False False False ..., False False False]
 [False  True False ...,  True  True False]
 ..., 
 [False  True False ..., False False False]
 [False  True  True ..., False False False]
 [False  True False ..., False  True False]]


In [8]:
def generate_column_feature(clean, dirty, number_distinct_values_per_column, \
                            avg_length1, max_length1, distinct_values_per_column):
    column_id = np.array([0] * len(clean) * len(clean.columns))
    column_distinct_values = np.array([0] * len(clean) * len(clean.columns))
    column_avg_length = np.array([0.0] * len(clean) * len(clean.columns))
    column_max_length = np.array([0] * len(clean) * len(clean.columns))
    current_value_fraction = np.array([0.0] * len(clean) * len(clean.columns))
    current_value_length = np.array([0] * len(clean) * len(clean.columns))
    current_row_avg_fraction = np.array([0.0] * len(clean) * len(clean.columns))
    current_row_avg_length = np.array([0.0] * len(clean) * len(clean.columns))
    
    i = 0
    for row in range(len(clean)):
        
        row_sum_length = 0.0
        row_sum_fraction = 0.0
        for column in range(len(clean.columns)):
            row_sum_fraction += distinct_values_per_column[column].get(dirty.values[row][column], 0)
            row_sum_length += len(str(dirty.values[row][column]))
        row_avg_length = row_sum_length / len(clean.columns)
        row_avg_fraction = row_sum_fraction / len(clean.columns)
            
        
        for column in range(len(clean.columns)):
            # attribute based
            column_id[i] = column
            column_distinct_values[i] = number_distinct_values_per_column[column]
            column_avg_length[i] = avg_length1[column]
            column_max_length[i] = max_length1[column]
            # value based
            current_value_fraction[i] = distinct_values_per_column[column].get(dirty.values[row][column], 0)
            current_value_length[i] = len(str(dirty.values[row][column]))
            # row based
            current_row_avg_fraction[i] = row_avg_fraction
            current_row_avg_length[i] = row_avg_length
            
            i += 1

    schema = ["col_id","col_distinct_values","col_avg_length","col_max_length",\
              "val_fraction","val_length","row_avg_fraction","row_avg_length"]

    return [column_id, column_distinct_values, column_avg_length, column_max_length, \
            current_value_fraction, current_value_length, \
            current_row_avg_fraction, current_row_avg_length], schema

In [9]:
def generate_target(clean, ground_truth):
    target = np.array([False] * len(clean) * len(clean.columns))
    
    i = 0
    for row in range(len(clean)):
        for column in range(len(clean.columns)):
            target[i] = ground_truth[row,column]
            i += 1
            
    return target

In [10]:
features = []

column_features, schema_metadata = generate_column_feature(clean, dirty, \
                                          number_distinct_values_per_column, \
                                          avg_length, max_length, distinct_values_per_column)
for i in range(len(column_features)):
    features.append(column_features[i])

In [11]:
total_schema = []
#total_schema.extend(schema_tools)
total_schema.extend(schema_metadata)
print total_schema

['col_id', 'col_distinct_values', 'col_avg_length', 'col_max_length', 'val_fraction', 'val_length', 'row_avg_fraction', 'row_avg_length']


In [12]:
target = generate_target(clean, ground_truth)

In [13]:
feature_matrix = np.column_stack(features)

In [14]:
print feature_matrix.shape

(1131672, 8)


In [15]:
def create_data(feature_matrix, train_size = 0.00001):
    train_len = train_size
    test_len = feature_matrix.shape[0] - train_len
    
    if (train_size <= 1):
        train_len = int(math.ceil(feature_matrix.shape[0] * train_size))
        test_len = int(math.floor(feature_matrix.shape[0] * (1 - train_size)))
    
    msk = np.ones((train_len,))
    msk = np.concatenate((msk, np.zeros((test_len,))))
    msk = msk == 1.
    np.random.shuffle(msk)
    
    indices_true = np.where(msk)[0]
    indices_false = np.where(~msk)[0]

    train = feature_matrix[indices_true,:]
    train_target = target[msk]
    test = feature_matrix[indices_false,:]
    test_target = target[~msk]
    
    return train, train_target, test, test_target

In [16]:
def create_next_data(train, train_target, feature_matrix, target, y_pred, n):
    diff = np.absolute(y_pred - 0.5)
    sorted_ids = np.argsort(diff)
        
    plt.hist(diff)
    plt.show()
    
    for i in range(n):
        train = sparse.vstack((train, feature_matrix[sorted_ids[i]]))
        train_target = np.append(train_target, [target[sorted_ids[i]]])
        
    print train.shape
        
    return train, train_target

In [18]:
input_file = "/tmp/test.csv"
output_file = "/tmp/rules.csv"
number_rules = 200

dirty.to_csv(input_file,header=True, index=False)

In [19]:
with open("rule_features4", "rb") as f:
    rule_decision = pickle.load(f)
    f.close

with open("rule_names4", "rb") as g:
    rules_names = pickle.load(g)
    f.close

In [20]:
feature_matrix_all = sparse.hstack((feature_matrix,rule_decision))

In [21]:
feature_matrix_all = feature_matrix_all.tocsr()

In [22]:
#feature_matrix_all = feature_matrix
print feature_matrix_all.shape

(1131672, 208)


In [23]:
total_schema_all = []
total_schema_all.extend(schema_metadata)
total_schema_all.extend(rules_names)

In [24]:
for nn in range(len(total_schema_all)):
    total_schema_all[nn] = str(nn) + "_" + total_schema_all[nn]

In [25]:
print total_schema_all

['0_col_id', '1_col_distinct_values', '2_col_avg_length', '3_col_max_length', '4_val_fraction', '5_val_length', '6_row_avg_fraction', '7_row_avg_length', '8_I=> ', '9_7=> ', '10_3=> ', '11_2=> ', '12_1=> ', '13_0=> ', '14_ =>A', '15_ =>E', '16_ =>R', '17_ =>D', '18_4=> ', '19_ =>S', '20_A=> ', '21_5=> ', '22_ =>e', '23_ =>a', '24_ =>T', '25_ =>r', '26_E=> ', '27_6=> ', '28_a=> ', '29_ =>t', '30_ =>N', '31_e=> ', '32_R=> ', '33_ =>L', '34_N=> ', '35_ =>I', '36_ =>O', '37_T=> ', '38_ =>AE', '39_ =>n', '40_S=> ', '41_ =>AR', '42_ =>ER', '43_r=> ', '44_8=> ', '45_t=> ', '46_1=> A', '47_1=>A', '48_n=> ', '49_ 1=>A', '50_9=> ', '51_O=> ', '52_ =>er', '53_12=> ', '54_ =>o', '55_ =>C', '56_01=> ', '57_ =>i', '58_1=> R', '59_L=> ', '60_1=>0', '61_ =>DR', '62_1=>R', '63_ 1=>R', '64_o=> ', '65_ =>ae', '66_ =>AT', '67_ =>d', '68_ =>l', '69_ =>AN', '70_ =>ar', '71_1=>E', '72_1=> 0', '73_ =>ET', '74_1=> E', '75_ =>RT', '76_D=> ', '77_l=> ', '78_1=>T', '79_i=> ', '80_ =>s', '81_1=>S', '82_ 1=>E', '83

In [26]:
ff = feature_matrix_all[:,0]
row, col, data = sparse.find(ff == 4)
print row
#print address_id
feature_matrix_all_address = feature_matrix_all[row,:]

[      4      16      28 ..., 1131640 1131652 1131664]


In [27]:
feature_matrix_all_save = feature_matrix_all
feature_matrix_all = feature_matrix_all_address

In [28]:
target_save = target
target = target[row]

In [29]:
feature_matrix_all_address

<94306x208 sparse matrix of type '<type 'numpy.float64'>'
	with 4534363 stored elements in Compressed Sparse Row format>

In [ ]:
feature_matrix_all

In [ ]:
train_size = 1000

our_params = {'eta': 0.1, 'seed': 0, 'subsample': 0.8, 'colsample_bytree': 0.8,
                          'objective': 'binary:logistic', 'max_depth': 3, 'min_child_weight': 1}


train, train_target, test, test_target = create_data(feature_matrix_all, train_size)
xgdmat = xgb.DMatrix(train, train_target, feature_names=total_schema_all)

cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5]}
ind_params = {'max_depth': 3, 'learning_rate': 0.1, 'n_estimators': 1000, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
     'objective': 'binary:logistic'}

optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                    cv_params, 
                     scoring = 'f1', cv = 5, n_jobs = 4) 

optimized_GBM.fit(train, train_target)

print "best scores: " + str(optimized_GBM.grid_scores_)

our_params = optimized_GBM.best_params_

print "train size: " + str(train_size) + "best params: " + str(our_params)

In [30]:
f_scores = []
precision_scores = []
recall_scores = []

models = dict()

n = 5
for train_size in [1000]:
    f_current = 0.0
    precision_current = 0.0
    recall_current = 0.0
    
    our_params = {'eta': 0.1, 'seed': 0, 'subsample': 0.8, 'colsample_bytree': 0.8,
                          'objective': 'binary:logistic', 'max_depth': 5, 'min_child_weight': 1}

    for t in range(n):
        train, train_target, test, test_target = create_data(feature_matrix_all, train_size)
        
        print train.shape
        
        xgdmat = xgb.DMatrix(train, train_target, feature_names=total_schema_all)
 
        final_gb = xgb.train(our_params, xgdmat, num_boost_round=3000)

        testdmat = xgb.DMatrix(feature_matrix_all, feature_names=total_schema_all)
        y_pred = final_gb.predict(testdmat)

        res = (y_pred > 0.5)

        print f1_score(target, res)
        f_current += f1_score(target, res)
        precision_current += precision_score(target, res)
        recall_current += recall_score(target, res)
        
        print final_gb.get_fscore()
    
    f_scores.append(f_current / n)
    precision_scores.append(precision_current / n)
    recall_scores.append(recall_current / n)

print "F-Score: " + str(f_scores)
print "Precision: " + str(precision_scores)
print "Recall: " + str(recall_scores)

(1000, 1207)
0.0
{'116_a=>r': 12, '0_col_id': 1093, '611_F=>L': 1, '921_a=> ': 3, '467_ =>l': 2, '357_O=>S': 8, '359_l=>a': 5, '823_P=> O': 8, '316_e=>r': 4, '423_ =>T': 12, '352_e=>a': 2, '315_a=>e': 1, '511_A=>N': 1, '556_T=>O': 7, '717_ =>3': 3, '810_O=> ': 8, '945_,=> A': 2, '320_r=>e': 5, '720_A=> ': 1, '410_2=> ': 1, '1008_0=>-': 7, '716_ =>2': 1, '408_7=> ': 1, '943_O=> ': 4, '576_C=>A': 11, '413_ =>A': 15, '318_a=>r': 15, '915_A=> ': 22, '123_A=>L': 6, '813_P=> ': 1, '373_S=>A': 2, '1031_1=>-': 1, '380_A=>T': 3, '453_ =>o': 3, '818_o=> ': 7, '338_a=>l': 3, '363_T=>E': 9, '377_M=>E': 1, '5_val_length': 2036, '510_O=>N': 20, '508_A=>O': 1, '323_L=>E': 13, '420_5=> ': 1, '819_B=>O': 2, '349_L=>A': 4, '309_R=>A': 4, '955_T=> ': 1, '130_e=>r': 5, '428_ =>t': 10, '828_O=>X': 1, '120_a=>e': 9, '482_A=>E': 1, '933_A=> ,': 7, '332_R=>O': 2, '313_a=>n': 1, '215_a=>n': 1, '321_o=>n': 13, '977_,o=> ': 1, '415_ =>R': 23, '821_ =>3': 1, '1007_5=>0': 1, '1_col_distinct_values': 840, '346_R=>I

In [75]:
final_gb.feature_names

['0_col_id',
 '1_col_distinct_values',
 '2_col_avg_length',
 '3_col_max_length',
 '4_val_fraction',
 '5_val_length',
 '6_row_avg_fraction',
 '7_row_avg_length',
 '8_9=>47',
 '9_9=>78',
 '10_9=>4',
 '11_9=>6',
 '12_A=>79',
 '13_A=>9',
 '14_A=>89',
 '15_9=>7',
 '16_9=>8',
 '17_A=>69',
 '18_9=>5',
 '19_A=>59',
 '20_A=>49',
 '21_9=>3',
 '22_A=>39',
 '23_A=>19',
 '24_A=>29',
 '25_9=>1',
 '26_A=>09',
 '27_9=>2',
 '28_9=>0',
 '29_A=>789',
 '30_9=>57',
 '31_9=>67',
 '32_A=>679',
 '33_A=>689',
 '34_9=>68',
 '35_A=>589',
 '36_A=>579',
 '37_9=>56',
 '38_9=>58',
 '39_A=>569',
 '40_A=>489',
 '41_A=>479',
 '42_9=>48',
 '43_9=>14',
 '44_9=>16',
 '45_A=>179',
 '46_9=>45',
 '47_9=>46',
 '48_A=>469',
 '49_A=>459',
 '50_A=>289',
 '51_A=>189',
 '52_A=>389',
 '53_A=>279',
 '54_9=>17',
 '55_9=>18',
 '56_9=>37',
 '57_A=>379',
 '58_9=>38',
 '59_9=>27',
 '60_A=>169',
 '61_A=>369',
 '62_9=>28',
 '63_A=>269',
 '64_A=>159',
 '65_A=>359',
 '66_A=>259',
 '67_9=>15',
 '68_9=>35',
 '69_9=>36',
 '70_9=>26',
 '71_A=>14

In [26]:
importances = final_gb.get_fscore()
importances

{'0_col_id': 1070,
 '1008_0=>-': 12,
 '1014_1=>3': 1,
 '1020_0=>9': 1,
 '1023_0=>6': 1,
 '1031_1=>-': 5,
 '1032_9=>1': 1,
 '1062_1=>27': 1,
 '108_H=>A': 5,
 '109_E=>A': 7,
 '110_A=>E': 12,
 '112_A=>N': 8,
 '115_R=>A': 6,
 '130_e=>r': 14,
 '134_M=>A': 1,
 '140_e=>n': 4,
 '1_col_distinct_values': 1048,
 '2_col_avg_length': 628,
 '309_R=>A': 10,
 '311_E=>R': 11,
 '313_a=>n': 8,
 '314_R=>E': 4,
 '315_a=>e': 1,
 '317_A=>R': 5,
 '318_a=>r': 7,
 '320_r=>e': 1,
 '323_L=>E': 9,
 '327_e=>n': 9,
 '330_A=>L': 1,
 '339_E=>S': 3,
 '347_e=>s': 3,
 '366_r=>s': 4,
 '367_C=>A': 3,
 '369_L=>O': 1,
 '3_col_max_length': 462,
 '407_I=> ': 1,
 '411_1=> ': 12,
 '412_0=> ': 1,
 '413_ =>A': 10,
 '414_ =>E': 17,
 '415_ =>R': 9,
 '416_ =>D': 1,
 '418_ =>S': 2,
 '419_A=> ': 5,
 '421_ =>e': 8,
 '422_ =>a': 4,
 '423_ =>T': 7,
 '424_ =>r': 2,
 '428_ =>t': 7,
 '429_ =>N': 2,
 '434_ =>I': 2,
 '435_ =>O': 3,
 '450_O=> ': 1,
 '465_ =>AT': 1,
 '4_val_fraction': 3583,
 '508_A=>O': 10,
 '509_A=>L': 4,
 '511_A=>N': 2,
 '512_

In [30]:
#active learning

f_scores = []
precision_scores = []
recall_scores = []

models = dict()

n = 1
for train_size in [10]:
    f_current = 0.0
    precision_current = 0.0
    recall_current = 0.0
    
    our_params = {'eta': 0.1, 'seed': 0, 'subsample': 0.8, 'colsample_bytree': 0.8,
                          'objective': 'binary:logistic', 'max_depth': 5, 'min_child_weight': 1}

    print "train size: " + str(train_size) + "best params: " + str(our_params)
    
    for t in range(n):
        train, train_target, test, test_target = create_data(feature_matrix_all, train_size)
        
        xgdmat = xgb.DMatrix(train, train_target, feature_names=total_schema_all)
 
        final_gb = xgb.train(our_params, xgdmat, num_boost_round=3000)

        testdmat = xgb.DMatrix(feature_matrix_all, feature_names=total_schema_all)
        y_pred = final_gb.predict(testdmat)

        res = (y_pred > 0.5)
                
        print "1 - F-Score: " + str(f1_score(target, res))
        print "1 - Precision: " + str(precision_score(target, res))
        print "1 - Recall: " + str(recall_score(target, res))
        
        
        for r in range (9):
            train, train_target = create_next_data(train, train_target,feature_matrix_all, target, y_pred, 10)

            xgdmat2 = xgb.DMatrix(train, train_target, feature_names=total_schema_all)

            final_gb2 = xgb.train(our_params, xgdmat2, num_boost_round=3000)

            y_pred = final_gb2.predict(testdmat)

            res2 = (y_pred > 0.5)

            print "2 - F-Score: " + str(f1_score(target, res2))
            print "2 - Precision: " + str(precision_score(target, res2))
            print "2 - Recall: " + str(recall_score(target, res2))
                
        f_current += f1_score(target, res2)
        precision_current += precision_score(target, res2)
        recall_current += recall_score(target, res2)
    
    f_scores.append(f_current / n)
    precision_scores.append(precision_current / n)
    recall_scores.append(recall_current / n)

print "F-Score: " + str(f_scores)
print "Precision: " + str(precision_scores)
print "Recall: " + str(recall_scores)

train size: 10best params: {'colsample_bytree': 0.8, 'min_child_weight': 1, 'subsample': 0.8, 'eta': 0.1, 'objective': 'binary:logistic', 'seed': 0, 'max_depth': 5}
1 - F-Score: 0.634506541305
1 - Precision: 0.49816062648
1 - Recall: 0.873613388124
(20, 208)
2 - F-Score: 0.897858395643
2 - Precision: 0.980074061012
2 - Recall: 0.828368854728
(30, 208)
2 - F-Score: 0.870252139718
2 - Precision: 0.952599194794
2 - Recall: 0.801009219239
(40, 208)
2 - F-Score: 0.893657758277
2 - Precision: 0.840470429351
2 - Recall: 0.95403155407
(50, 208)
2 - F-Score: 0.955989154128
2 - Precision: 0.954862143676
2 - Recall: 0.957118828113
(60, 208)
2 - F-Score: 0.962529857721
2 - Precision: 0.939541390426
2 - Recall: 0.98667149275
(70, 208)
2 - F-Score: 0.954685253026
2 - Precision: 0.941511387164
2 - Recall: 0.96823301467
(80, 208)
2 - F-Score: 0.972304554291
2 - Precision: 0.970273937749
2 - Recall: 0.974343688121
(90, 208)
2 - F-Score: 0.989162204254
2 - Precision: 0.991149874944
2 - Recall: 0.9871824

In [2]:
%matplotlib inline  
importance_frame = pd.DataFrame({'Importance': list(importances.values()), 'Feature': list(importances.keys())})
importance_frame.sort_values(by = 'Importance', inplace = True)
importance_frame[0:40].plot(kind = 'barh', x = 'Feature', figsize = (8,8), color = 'orange')

NameError: name 'pd' is not defined

In [1]:
importances = final_gb2.get_fscore()

import operator
sorted_imp = sorted(importances.items(), key=operator.itemgetter(1), reverse=True)
print sorted_imp


NameError: name 'final_gb2' is not defined

In [24]:
f_scores = []
precision_scores = []
recall_scores = []

n = 5
for train_size in [2, 5, 10, 15, 100, 1000, 10000, 100000, 200000]:
    f_current = 0.0
    precision_current = 0.0
    recall_current = 0.0
    for t in range(n):
        train, train_target, test, test_target = create_data(feature_matrix, train_size)

        gnb = GaussianNB()
        final = gnb.fit(train, train_target)

        res = final.predict(test)
        
        f_current += f1_score(test_target, res)
        precision_current += precision_score(test_target, res)
        recall_current += recall_score(test_target, res)
    
    f_scores.append(f_current / n)
    precision_scores.append(precision_current / n)
    recall_scores.append(recall_current / n)

print "F-Score: " + str(f_scores)
print "Precision: " + str(precision_scores)
print "Recall: " + str(recall_scores)

F-Score: [0.1890476877457227, 0.37834691631269701, 0.58013839104177711, 0.64066265622150431, 0.76637150886752126, 0.88541243966779748, 0.8904424312141298, 0.89418077951191766, 0.89400388396576158]
Precision: [0.12948940111400137, 0.62960554503886479, 0.70424754442742199, 0.64042826713778922, 0.807455474301358, 0.90216814718046978, 0.89283842140448699, 0.89833355466123843, 0.89884032575999573]
Recall: [0.35217309570185013, 0.34439729061364038, 0.55682396832813552, 0.65127321356131895, 0.75088111053690398, 0.86981633402841962, 0.88823521575000952, 0.89007312570900476, 0.88922014444919262]


In [25]:
f_scores = []
precision_scores = []
recall_scores = []

n = 5
for train_size in [30,100,1000,10000,30000,100000]:
    f_current = 0.0
    precision_current = 0.0
    recall_current = 0.0
    
    for t in range(n):
        train, train_target, test, test_target = create_data(feature_matrix, train_size)

        logistic = linear_model.LogisticRegression()
        final = logistic.fit(train, train_target)

        res = final.predict(test)
        
        f_current += f1_score(test_target, res)
        precision_current += precision_score(test_target, res)
        recall_current += recall_score(test_target, res)
    
    f_scores.append(f_current / n)
    precision_scores.append(precision_current / n)
    recall_scores.append(recall_current / n)

print "F-Score: " + str(f_scores)
print "Precision: " + str(precision_scores)
print "Recall: " + str(recall_scores)

F-Score: [0.87223511722492031, 0.89280466393892388, 0.9687255651065676, 0.95227522685165267, 0.95391058569075116, 0.95623387247360725]
Precision: [0.93076489020254838, 0.90334381529741548, 0.99457415845025388, 0.9703945073953697, 0.97138393496406006, 0.97921391826144721]
Recall: [0.84483747697020684, 0.88643684568142933, 0.94420199829901663, 0.93495260626051502, 0.93739386301633743, 0.93440290289820849]


In [145]:
recall = recall_score(test_target, res)
print "recall: " + str(recall)

recall: 0.931965062081


In [146]:
precision = precision_score(test_target, res)
print "precision: " + str(precision)

precision: 0.963111878677


In [99]:
column_detect = np.array([False] * len(clean) * len(clean.columns))
target = np.array([False] * len(clean) * len(clean.columns))

In [100]:
i = 0
for row in range(len(clean)):
    for column in range(len(clean.columns)):
        column_detect[i] = (row, column) in error_sets[4]
        target[i] = ground_truth[row,column]
        i += 1

In [101]:
TP = np.sum(np.logical_and(column_detect == True, target == True))

In [102]:
FP = np.sum(np.logical_and(column_detect == True, target == False))

In [103]:
FN = np.sum(np.logical_and(column_detect == False, target == True))

In [104]:
print "precision: " + str(float(TP) / (TP + FP))

precision: 0.999349493101


In [105]:
print "recall: " + str(float(TP) / (TP + FN))

recall: 0.946619217082
